In [70]:
#procurando PySpark no sistema
import findspark
findspark.init('./.linuxbrew/Cellar/apache-spark/2.4.4/libexec')
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [71]:
findspark.find()

'./.linuxbrew/Cellar/apache-spark/2.4.4/libexec'

In [72]:
# SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("Semantix") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [82]:
import glob
untz_data_files = glob.glob('*.gz')
df = spark.read.text(untz_data_files)
#nao ha um esquema definido
df.printSchema()


root
 |-- value: string (nullable = true)



In [109]:
untz_data_files

['NASA_Jul95.gz', 'NASA_Aug95.gz']

In [85]:
type(df)

pyspark.sql.dataframe.DataFrame

In [88]:
rdd = df.rdd
type(rdd)


pyspark.rdd.RDD

In [89]:
df.show(2, truncate = False)

+-------------------------------------------------------------------------------------------------+
|value                                                                                            |
+-------------------------------------------------------------------------------------------------+
|199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245           |
|unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985|
+-------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [90]:
rdd.take(6)

[Row(value='199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245'),
 Row(value='unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985'),
 Row(value='199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085'),
 Row(value='burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0'),
 Row(value='199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179'),
 Row(value='burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0')]

## Tipando sever logs: remotehost rfc931 authuser [date] "request" status bytes

- Primeira coluna: Remote Host
- Segunda Coluna: RFC931
- Terceira Coluna: authuser	
- Quarta Coluna: [date]	(timestamp)
- Quinta Coluna: "request"	
- Sexta Coluna: Status (200 OK, 404, 500, 400)
- Setima Coluna: Bytes


In [152]:
import re
logs = [item['value'] for item in df.take(10)]
logs

['199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245',
 'unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985',
 '199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085',
 'burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0',
 '199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179',
 'burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0',
 'burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/video/livevideo.gif HTTP/1.0" 200 0',
 '205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/countdown.html HTTP/1.0" 200 3985',
 'd104.aa.net - - [01/Jul/1995:00:00:13 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985',
 '129.94.144.152 - - [01/Jul/

In [113]:
#pattern pegando strings com pontos 
regex_host = r'(^\S+\.[\S+\.]+\S+)\s'
remote_hosts = []
for item in logs:
    if re.search(regex_host, item):
        remote_hosts.append(re.search(regex_host, item).group(1))
    else:
        'nope'
#list de remote_hosts dentro da lista de itens    
remote_hosts[1]

'unicomp6.unicomp.net'

In [111]:
#pattern pegando datas por dupla de digitos (ou 4 no caso do ano)
timestamp_pattern = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'
data = []
for item in logs:
    data.append(re.search(timestamp_pattern, item).group(1))
    
data[1]

'01/Jul/1995:00:00:06 -0400'

In [115]:
#pegando tuplas em trio, pois a requisicao conta com um metodo, informacao do dado e versao http
request_pattern = r'\"(\S+)\s(\S+)\s*(\S*)\"'
request = []
for item in logs:
    if re.search(request_pattern, item):
        request.append(re.search(request_pattern, item).groups())
    else:
        'nope'
request[1]

('GET', '/shuttle/countdown/', 'HTTP/1.0')

In [116]:
#pegando digitos sem espaco, referente ao response
response_pattern = r'\s(\d{3})\s'
response = []
for item in logs:
    response.append(re.search(response_pattern, item).group(1))
response[1]    


'200'

In [118]:
#bytes, regex com quantifier dos digitos
bytes_pattern = r'\s(\d+)$'
bytes_by = []
for item in logs:
    bytes_by.append(re.search(bytes_pattern, item).group(1))
bytes_by[1]    


'3985'

## Juntando tudo com regexp_extract

In [119]:
from pyspark.sql.functions import regexp_extract

df_geral = df.select(regexp_extract('value', regex_host, 1).alias('host'),
                         regexp_extract('value', timestamp_pattern, 1).alias('data'),
                         regexp_extract('value', request_pattern, 1).alias('metodo'),
                         regexp_extract('value', request_pattern, 2).alias('info'),
                         regexp_extract('value', request_pattern, 3).alias('versao http'),
                         regexp_extract('value', response_pattern, 1).cast('integer').alias('response'),
                         regexp_extract('value', bytes_pattern, 1).cast('integer').alias('bytes'))
df_geral.show(10, truncate=True)


+--------------------+--------------------+------+--------------------+-----------+--------+-----+
|                host|                data|metodo|                info|versao http|response|bytes|
+--------------------+--------------------+------+--------------------+-----------+--------+-----+
|        199.72.81.55|01/Jul/1995:00:00...|   GET|    /history/apollo/|   HTTP/1.0|     200| 6245|
|unicomp6.unicomp.net|01/Jul/1995:00:00...|   GET| /shuttle/countdown/|   HTTP/1.0|     200| 3985|
|      199.120.110.21|01/Jul/1995:00:00...|   GET|/shuttle/missions...|   HTTP/1.0|     200| 4085|
|  burger.letters.com|01/Jul/1995:00:00...|   GET|/shuttle/countdow...|   HTTP/1.0|     304|    0|
|      199.120.110.21|01/Jul/1995:00:00...|   GET|/shuttle/missions...|   HTTP/1.0|     200| 4179|
|  burger.letters.com|01/Jul/1995:00:00...|   GET|/images/NASA-logo...|   HTTP/1.0|     304|    0|
|  burger.letters.com|01/Jul/1995:00:00...|   GET|/shuttle/countdow...|   HTTP/1.0|     200|    0|
|     205.

In [120]:
print((df_geral.count(), len(df_geral.columns)))

(3461613, 7)


In [130]:
#count de null values
null_df = df_geral.filter(df_geral['host'].isNull()| 
                             df_geral['data'].isNull() | 
                             df_geral['metodo'].isNull() |
                             df_geral['info'].isNull() |
                             df_geral['versao http'].isNull() |
                             df_geral['response'].isNull()|
                             df_geral['bytes'].isNull())
null_df.count()

33905

In [134]:
#count de hosts unicos
unique_hosts = (df_geral.select('host').distinct().count())
unique_hosts
#poderia usar rdd, com rdd.key.distinct.count

137933

In [137]:
#count de erros 404
df_404 =(df_geral.filter(df_geral['response'] == 404))
df_404.count()

20899

In [138]:
#count de endpoints com maior numero de erros
mosterror_df = (df_404.groupBy('info').count().sort('count', ascending=False).limit(5))
mosterror_df.show(truncate=False)


+--------------------------------------------+-----+
|info                                        |count|
+--------------------------------------------+-----+
|/pub/winvn/readme.txt                       |2004 |
|/pub/winvn/release.txt                      |1732 |
|/shuttle/missions/STS-69/mission-STS-69.html|683  |
|/shuttle/missions/sts-68/ksc-upclose.gif    |428  |
|/history/apollo/a-001/a-001-patch-small.gif |384  |
+--------------------------------------------+-----+



In [141]:
#contagem de erros por dia, apenas os primeiros 20 primeiros valores
erros_daily_df = (df_404.groupBy('data').count().sort('count', ascending=False))
erros_daily_df.show(truncate=False)

+--------------------------+-----+
|data                      |count|
+--------------------------+-----+
|28/Aug/1995:11:56:35 -0400|7    |
|11/Aug/1995:12:05:59 -0400|7    |
|12/Jul/1995:10:35:09 -0400|5    |
|12/Jul/1995:10:24:50 -0400|5    |
|11/Aug/1995:12:05:58 -0400|5    |
|17/Aug/1995:16:55:00 -0400|5    |
|28/Aug/1995:17:14:32 -0400|5    |
|12/Jul/1995:10:35:11 -0400|5    |
|12/Jul/1995:10:21:30 -0400|5    |
|12/Jul/1995:10:20:43 -0400|5    |
|12/Jul/1995:10:35:12 -0400|5    |
|28/Aug/1995:17:14:42 -0400|5    |
|11/Jul/1995:14:08:06 -0400|5    |
|28/Aug/1995:17:14:28 -0400|4    |
|12/Jul/1995:10:35:03 -0400|4    |
|20/Jul/1995:07:21:17 -0400|4    |
|28/Aug/1995:01:05:47 -0400|4    |
|13/Aug/1995:14:28:59 -0400|4    |
|12/Jul/1995:10:21:32 -0400|4    |
|04/Aug/1995:18:45:58 -0400|4    |
+--------------------------+-----+
only showing top 20 rows



In [151]:
#evitando groupby para os bytes
rdd_geral = df_geral.rdd
#dividindo por 10e-9 para GB
bytes_data = rdd_geral.map(lambda x: x[4] / 1000000000.0)
bytes = bytes_data.sum()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 58.0 failed 1 times, most recent failure: Lost task 0.0 in stage 58.0 (TID 1252, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/devjoao/.linuxbrew/Cellar/apache-spark/2.4.4/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/devjoao/.linuxbrew/Cellar/apache-spark/2.4.4/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "./.linuxbrew/Cellar/apache-spark/2.4.4/libexec/python/pyspark/rdd.py", line 2499, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "./.linuxbrew/Cellar/apache-spark/2.4.4/libexec/python/pyspark/rdd.py", line 2499, in pipeline_func
    return func(sp